# Classification for Machine Learning




In [1]:
import sys
sys.path.insert(1, '../../crystal-code-tools/crystal_functions/crystal_functions/')

from file_readwrite import Crystal_output, write_cry_properties, Crystal_input, write_cry_gui,write_cry_input, Density
from execute import runcry
from convert import cry_gui2pmg

from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.io.cif import CifWriter
from ase.visualize import view

from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core.surface import SlabGenerator
import numpy as np
import pandas as pd

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
new_atom = 'Ca'

### Read the structures from the output files and replace with new_atom
This cell generates the structures list. This contains all the 4023 structures as pymatgen structure objects

In [67]:
cry_output = Crystal_output('data/classification/ml/mgo_confrand.out')
cry_output.config_analysis()


original_structure = cry_gui2pmg('data/classification/ml/mgo_confrand.gui')
structures = []
mg_atoms = []
ca_atoms = []
for j,substitutions in enumerate(cry_output.atom_type2):
    new_structure = original_structure.copy()
    for i in substitutions:
        new_structure.replace(i-1,new_atom)
    structures.append(new_structure)
    mg_atoms.append((np.array(cry_output.atom_type1[j])-1).tolist())
    ca_atoms.append((np.array(cry_output.atom_type2[j])-1).tolist())

### Shells

In [26]:
shells = np.unique(np.round(structures[0].distance_matrix[0],decimals=6),return_counts=True)[0].tolist()
display(shells)
np.unique(np.round(structures[0].distance_matrix[0],decimals=6),return_counts=True)[1]

[0.0,
 2.105,
 2.97692,
 3.645967,
 4.21,
 4.706923,
 5.156176,
 5.953839,
 6.315,
 6.656594,
 6.981495,
 7.291934,
 7.589685,
 7.876189,
 8.679137,
 8.930759,
 9.175482,
 9.873325,
 10.937901]

array([ 1,  6, 12,  8,  6, 24, 24, 12, 27, 12, 12,  8, 12, 24, 12,  3,  6,
        6,  1])

In [22]:
view(AseAtomsAdaptor().get_atoms(structures[0]))

<Popen: returncode: None args: ['/Users/brunocamino/miniconda3/envs/test_env...>

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


### Site surrounding

#### Second shell

In [7]:
unique_atom_type = []

# The magic happens here

# Create a list of the unique densities

In [48]:

#structure = cry_gui2pmg('data/classification/LTS_2_B3LYP_Ahl_b.gui')


#structure = cry_gui2pmg('data/classification/mgo_scel.gui')
structure = structures[0]

#How many first/second neighbours can an atom have?
max_n_first_shell = 6
max_n_second_shell = 12
max_n_third_shell = 8

#This is the final one (list of lists)

#list of lists. Each element has 0 central atom specie, 1: surrounding atom species (-1 if there are not enough neighbours
#for example: [8, 12, 12, 12, -1]. Unique elements only (unique to all structures)
unique_neighbours = []

#list of lists. Each element has 0 central atom specie, 1: surrounding atom species (-1 if there are not enough neighbours
#for example: [8, 12, 12, 12, -1]. All elements
first_neigh_atoms_all = []
#list of lists. Each element has 0 central atom index, 1: surrounding atom index (-1 if there are not enough neighbours
#All elements
first_neigh_all = []

#list of lists. Each element has 0 central atom specie, 1: surrounding atom species (-1 if there are not enough neighbours
#for example: [8, 12, 12, 12, -1]. Unique elements only (unique to this structure)
first_neigh_atoms_unique = []
#list of lists. Each element has 0 central atom index, 1: surrounding atom index (-1 if there are not enough neighbours
# Unique elements only (unique to this structure)
first_neigh_unique = []

#structure = mgo
#structure = structures[126]


for i,atom in enumerate(structure.sites):
    first_neigh = []
    first_neigh_atoms = []
    
    second_neigh = []
    second_neigh_atoms = []
    
    third_neigh = []
    third_neigh_atoms = []
    for j,atom in enumerate(structure.sites):        

        
        #Check is the atom j is in the first shell in the unit cell
        if abs(structure.sites[i].distance(structure.sites[j]) - shells[1]) < 0.1:
            first_neigh.append(j)
            first_neigh_atoms.append(structure.atomic_numbers[j])
        if abs(structure.sites[i].distance(structure.sites[j]) - shells[2]) < 0.1:
            second_neigh.append(j)
            second_neigh_atoms.append(structure.atomic_numbers[j])
    
        if abs(structure.sites[i].distance(structure.sites[j]) - shells[3]) < 0.1:
            third_neigh.append(j)
            third_neigh_atoms.append(structure.atomic_numbers[j])
            
    #print(i,j,first_neigh_atoms)
    sort_list = np.argsort(np.array(first_neigh_atoms))
    sort_list_second = np.argsort(np.array(second_neigh_atoms))
    sort_list_third = np.argsort(np.array(third_neigh_atoms))
    
    #First neighbour index for the i-th atom sorted
    first_neigh = (np.array(first_neigh)[sort_list]).tolist()
    #First neighbour species for the i-th atom sorted
    first_neigh_atoms = (np.array(first_neigh_atoms)[sort_list]).tolist()
    
    #Second neighbour index for the i-th atom sorted
    second_neigh = (np.array(second_neigh)[sort_list_second]).tolist()
    #Second neighbour species for the i-th atom sorted
    second_neigh_atoms = (np.array(second_neigh_atoms)[sort_list_second]).tolist()
    
    #T neighbour index for the i-th atom sorted
    third_neigh = (np.array(third_neigh)[sort_list_third]).tolist()
    #T neighbour species for the i-th atom sorted
    third_neigh_atoms = (np.array(third_neigh_atoms)[sort_list_third]).tolist()

    #Prepend the atom index to the first neighbours list
    first_neigh.insert(0,i)
    #Prepend the atom specie to the first neighbours list
    first_neigh_atoms.insert(0,structure.atomic_numbers[i])
    
    #Add -1 to first_neigh and first_neigh_atoms to reach the same lenght for all vectors
    if len(first_neigh) < max_n_first_shell+1:
        n = max_n_first_shell+1 - len(first_neigh)
        first_neigh.extend([-1]*n)
        first_neigh_atoms.extend([-1]*n)
    
    #Same for the second shell
    if len(second_neigh) < max_n_second_shell:
        n = max_n_second_shell+1 - len(second_neigh)
        second_neigh.extend([-1]*n)
        second_neigh_atoms.extend([-1]*n)
        
    #Same for the third shell
    if len(third_neigh) < max_n_third_shell:
        n = max_n_third_shell+1 - len(third_neigh)
        third_neigh.extend([-1]*n)
        third_neigh_atoms.extend([-1]*n)
        
    #TMP: append second to first neigh
    first_neigh.extend(second_neigh)
    first_neigh_atoms.extend(second_neigh_atoms)
    first_neigh.extend(third_neigh)
    first_neigh_atoms.extend(third_neigh_atoms)
    
    
    
    first_neigh_atoms_all.append(first_neigh_atoms) 
    first_neigh_all.append(first_neigh)
    print(first_neigh_atoms)
first_neigh_atoms_unique.extend(np.unique(first_neigh_atoms_all,axis=0,return_index=True)[0].tolist())
index = np.unique(first_neigh_atoms_all,axis=0,return_index=True)[1]
first_neigh_unique.extend((np.array(first_neigh_all)[index]).tolist())
              

[20, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 8, 8, 8, 8, 8, 8, 8, 8]
[20, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 8, 8, 8, 8, 8, 8, 8, 8]
[20, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 8, 8, 8, 8, 8, 8, 8, 8]
[20, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 8, 8, 8, 8, 8, 8, 8, 8]
[20, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 8, 8, 8, 8, 8, 8, 8, 8]
[20, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 8, 8, 8, 8, 8, 8, 8, 8]
[20, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 8, 8, 8, 8, 8, 8, 8, 8]
[20, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 8, 8, 8, 8, 8, 8, 8, 8]
[20, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 8, 8, 8, 8, 8, 8, 8, 8]
[20, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 8, 8, 8, 8, 8, 8, 8, 8]
[20, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12,

[12, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12, 20, 20, 20, 20, 8, 8, 8, 8, 8, 8, 8, 8]
[12, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12, 20, 20, 20, 20, 8, 8, 8, 8, 8, 8, 8, 8]
[12, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12, 20, 20, 20, 20, 8, 8, 8, 8, 8, 8, 8, 8]
[12, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12, 20, 20, 20, 20, 8, 8, 8, 8, 8, 8, 8, 8]
[12, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12, 20, 20, 20, 20, 8, 8, 8, 8, 8, 8, 8, 8]
[12, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12, 20, 20, 20, 20, 8, 8, 8, 8, 8, 8, 8, 8]
[12, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12, 20, 20, 20, 20, 8, 8, 8, 8, 8, 8, 8, 8]
[12, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12, 20, 20, 20, 20, 8, 8, 8, 8, 8, 8, 8, 8]
[12, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12, 20, 20, 20, 20, 8, 8, 8, 8, 8, 8, 8, 8]
[12, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12, 20, 20, 20, 20, 8, 8, 8, 8, 8, 8, 8, 8]
[12, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12,

[8, 12, 12, 12, 12, 12, 20, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12]
[8, 12, 12, 12, 12, 12, 20, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12]
[8, 12, 12, 12, 12, 12, 20, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12]
[8, 12, 12, 12, 12, 12, 20, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12]
[8, 12, 12, 12, 12, 12, 20, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12]
[8, 12, 12, 12, 12, 12, 20, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12]
[8, 12, 12, 12, 12, 12, 20, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12]
[8, 12, 12, 12, 12, 12, 20, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12]
[8, 12, 12, 12, 12, 12, 20, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12]
[8, 12, 12, 12, 12, 12, 20, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12]
[8, 12, 12, 12, 12, 12, 20, 8, 8, 8, 8, 

In the 2D array first_neigh_atoms_all each row corresponds to an atom and the array contains:
- 0 = atom number of the atom whose neighbours I'm analysing
- 1:6 = atomic numbers of the atoms in the first coord shell
- 7:19 = atomic numbers of the atoms in the second coord shell
- 20:28 = atomic numbers of the atoms in the third coord shell

## Polar coordinates

In [208]:
def cart2sph(x, y, z):
    hxy = np.hypot(x, y)
    r = np.hypot(hxy, z)
    el = np.arctan2(z, hxy)
    az = np.arctan2(y, x)
    if np.around(az,6) ==  np.around(2*np.pi,6) \
    or np.around(az,6) ==  -np.around(2*np.pi,6):
        az = 0.
    if np.around(az,6) < 0.:
        az = np.round(2*np.pi+az,6)
    return [round(az,6), round(el,6), round(r,6)]

In [50]:
def cart2cent2sph(atom,center):
    x = atom[0]-center[0]
    y = atom[1]-center[1]
    z = atom[2]-center[2]
    return cart2sph(x, y, z)

In [54]:
for coord in structures[0].cart_coords:
    print(cart2cent2sph(coord,structures[0].cart_coords[0]))

[0.0, 0.0, 0.0]
[0.0, 1.570796, 4.21]
[0.0, -1.570796, 4.21]
[1.570796, 0.0, 4.21]
[1.570796, 0.785398, 5.953839]
[1.570796, -0.785398, 5.953839]
[-1.570796, 0.0, 4.21]
[-1.570796, 0.785398, 5.953839]
[-1.570796, -0.785398, 5.953839]
[0.0, 0.0, 4.21]
[0.0, 0.785398, 5.953839]
[0.0, -0.785398, 5.953839]
[0.785398, 0.0, 5.953839]
[0.785398, 0.61548, 7.291934]
[0.785398, -0.61548, 7.291934]
[-0.785398, 0.0, 5.953839]
[-0.785398, 0.61548, 7.291934]
[-0.785398, -0.61548, 7.291934]
[3.141593, 0.0, 4.21]
[3.141593, 0.785398, 5.953839]
[3.141593, -0.785398, 5.953839]
[2.356194, 0.0, 5.953839]
[2.356194, 0.61548, 7.291934]
[2.356194, -0.61548, 7.291934]
[-2.356194, 0.0, 5.953839]
[-2.356194, 0.61548, 7.291934]
[-2.356194, -0.61548, 7.291934]
[0.785398, 0.0, 2.97692]
[0.785398, 0.955317, 5.156176]
[0.785398, -0.955317, 5.156176]
[-1.249046, 0.0, 6.656594]
[-1.249046, 0.563943, 7.876189]
[-1.249046, -0.563943, 7.876189]
[-0.785398, 0.0, 2.97692]
[-0.785398, 0.955317, 5.156176]
[-0.785398, -0.9553

In [209]:
centered_sph_coords = []
centered_sph_coords_all = []
for j in range(structures[0].num_sites):
    centered_sph_coords = []
    for i in range(structures[0].num_sites):    
        structure_tmp = structures[0]

        translation_vector = structures[0].sites[j].distance_and_image(structures[0].sites[i])[1]


        new_cart_coords = structures[0].cart_coords[i]+(translation_vector*structures[0].lattice.abc)

        centered_cart_coords = new_cart_coords-structures[0].cart_coords[j] #This will be i

        centered_sph_coords.append(cart2sph(centered_cart_coords[0],centered_cart_coords[1],centered_cart_coords[2]))
        #print(structures[0].cart_coords[i],centered_cart_coords,cart2sph(centered_cart_coords[0],centered_cart_coords[1],centered_cart_coords[2]))

    centered_sph_coords_all.append(centered_sph_coords)
centered_sph_coords_all = np.array(centered_sph_coords_all)

In [210]:
order = np.argsort(centered_sph_coords_all[0][:,2])
centered_sph_coords_all[0][order]

array([[ 0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  2.105   ],
       [ 4.712389,  0.      ,  2.105   ],
       [ 0.      , -1.570796,  2.105   ],
       [ 1.570796,  0.      ,  2.105   ],
       [ 0.      ,  1.570796,  2.105   ],
       [ 3.141593,  0.      ,  2.105   ],
       [ 0.      ,  0.785398,  2.97692 ],
       [ 3.926991,  0.      ,  2.97692 ],
       [ 4.712389, -0.785398,  2.97692 ],
       [ 4.712389,  0.785398,  2.97692 ],
       [ 2.356194,  0.      ,  2.97692 ],
       [ 3.141593, -0.785398,  2.97692 ],
       [ 1.570796, -0.785398,  2.97692 ],
       [ 0.785398,  0.      ,  2.97692 ],
       [ 1.570796,  0.785398,  2.97692 ],
       [ 0.      , -0.785398,  2.97692 ],
       [ 5.497787,  0.      ,  2.97692 ],
       [ 3.141593,  0.785398,  2.97692 ],
       [ 0.785398,  0.61548 ,  3.645967],
       [ 0.785398, -0.61548 ,  3.645967],
       [ 5.497787,  0.61548 ,  3.645967],
       [ 5.497787, -0.61548 ,  3.645967],
       [ 3.926991, -0.61548 ,  3.6

In [211]:
r_unique = np.unique(centered_sph_coords_all[0][:,2])

In [212]:
atoms_shell = []
for unique in r_unique:
    atoms_shell.append(np.where(centered_sph_coords_all[0][:,2] == unique)[0].tolist())

In [157]:
atoms_shell

[[0],
 [108, 110, 162, 180, 189, 195],
 [27, 33, 45, 51, 54, 56, 72, 74, 81, 83, 87, 89],
 [135, 137, 141, 143, 153, 155, 159, 161],
 [1, 2, 3, 6, 9, 18],
 [111,
  113,
  114,
  116,
  117,
  119,
  126,
  128,
  163,
  164,
  165,
  168,
  181,
  182,
  183,
  186,
  190,
  191,
  196,
  197,
  198,
  204,
  207,
  213],
 [28,
  29,
  34,
  35,
  46,
  47,
  52,
  53,
  57,
  59,
  60,
  62,
  75,
  77,
  78,
  80,
  90,
  92,
  96,
  98,
  99,
  101,
  105,
  107],
 [4, 5, 7, 8, 10, 11, 12, 15, 19, 20, 21, 24],
 [109,
  120,
  122,
  123,
  125,
  129,
  131,
  132,
  134,
  166,
  167,
  169,
  170,
  171,
  184,
  185,
  187,
  188,
  192,
  199,
  200,
  205,
  206,
  208,
  209,
  214,
  215],
 [30, 36, 42, 48, 55, 63, 65, 73, 82, 84, 86, 88],
 [136, 138, 140, 142, 144, 146, 150, 152, 154, 156, 158, 160],
 [13, 14, 16, 17, 22, 23, 25, 26],
 [112, 115, 118, 127, 172, 173, 174, 177, 193, 194, 201, 210],
 [31,
  32,
  37,
  38,
  43,
  44,
  49,
  50,
  58,
  61,
  66,
  68,
  69,
 

In [230]:
a = centered_sph_coords_all[0][atoms_shell[2]]
a

array([[ 0.785398,  0.      ,  2.97692 ],
       [ 5.497787,  0.      ,  2.97692 ],
       [ 2.356194,  0.      ,  2.97692 ],
       [ 3.926991,  0.      ,  2.97692 ],
       [ 0.      ,  0.785398,  2.97692 ],
       [ 0.      , -0.785398,  2.97692 ],
       [ 3.141593,  0.785398,  2.97692 ],
       [ 3.141593, -0.785398,  2.97692 ],
       [ 1.570796,  0.785398,  2.97692 ],
       [ 1.570796, -0.785398,  2.97692 ],
       [ 4.712389,  0.785398,  2.97692 ],
       [ 4.712389, -0.785398,  2.97692 ]])

In [226]:
c = np.argsort(centered_sph_coords_all[0][atoms_shell[1]][:,1]*10 + centered_sph_coords_all[0][atoms_shell[1]][:,0])

In [231]:
c = np.argsort(centered_sph_coords_all[0][atoms_shell[2]][:,1]*10 + centered_sph_coords_all[0][atoms_shell[2]][:,0])

In [234]:
np.array(atoms_shell[2])[c]

array([56, 83, 74, 89, 27, 45, 51, 33, 54, 81, 72, 87])

In [233]:
np.array(structures[0].atomic_numbers)[np.array(atoms_shell[2])[c]]

array([12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12])

In [161]:
b = np.argsort(a[:,1])
a[b]

array([[ 0.      , -1.570796,  2.105   ],
       [ 0.      ,  0.      ,  2.105   ],
       [ 3.141593,  0.      ,  2.105   ],
       [ 1.570796,  0.      ,  2.105   ],
       [-1.570796,  0.      ,  2.105   ],
       [ 0.      ,  1.570796,  2.105   ]])

In [146]:
np.array(structures[0].atomic_numbers)[order]

array([20,  8,  8,  8,  8,  8,  8, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12,  8,  8,  8,  8,  8,  8,  8,  8, 20, 20, 20, 20, 20, 12,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 20, 12,
       20, 20, 20, 20, 12, 20, 20, 20,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8, 12, 12, 20, 12, 20, 20, 12, 20,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8, 12, 12, 12,  8,
        8,  8,  8,  8,  8, 12, 12, 12, 12, 12, 12,  8])

In [131]:
centered_sph_coords = []
for i in range(structures[0].num_sites):    
    structure_tmp = structures[0]

    translation_vector = structures[0].sites[0].distance_and_image(structures[0].sites[i])[1]

    
    new_cart_coords = structures[0].cart_coords[i]+(translation_vector*structures[0].lattice.abc)
    
    centered_cart_coords = new_cart_coords-structures[0].cart_coords[0] #This will be i
    
    centered_sph_coords.append(cart2sph(centered_cart_coords[0],centered_cart_coords[1],centered_cart_coords[2]))
    print(structures[0].cart_coords[i],centered_cart_coords,cart2sph(centered_cart_coords[0],centered_cart_coords[1],centered_cart_coords[2]))

centered_sph_coords = np.array(centered_sph_coords)

[0. 0. 0.] [0. 0. 0.] [0.0, 0.0, 0.0]
[0.   0.   4.21] [0.   0.   4.21] [0.0, 1.570796, 4.21]
[ 0.    0.   -4.21] [ 0.    0.   -4.21] [0.0, -1.570796, 4.21]
[0.   4.21 0.  ] [0.   4.21 0.  ] [1.570796, 0.0, 4.21]
[0.   4.21 4.21] [0.   4.21 4.21] [1.570796, 0.785398, 5.953839]
[ 0.    4.21 -4.21] [ 0.    4.21 -4.21] [1.570796, -0.785398, 5.953839]
[ 0.   -4.21  0.  ] [ 0.   -4.21  0.  ] [-1.570796, 0.0, 4.21]
[ 0.   -4.21  4.21] [ 0.   -4.21  4.21] [-1.570796, 0.785398, 5.953839]
[ 0.   -4.21 -4.21] [ 0.   -4.21 -4.21] [-1.570796, -0.785398, 5.953839]
[4.21 0.   0.  ] [4.21 0.   0.  ] [0.0, 0.0, 4.21]
[4.21 0.   4.21] [4.21 0.   4.21] [0.0, 0.785398, 5.953839]
[ 4.21  0.   -4.21] [ 4.21  0.   -4.21] [0.0, -0.785398, 5.953839]
[4.21 4.21 0.  ] [4.21 4.21 0.  ] [0.785398, 0.0, 5.953839]
[4.21 4.21 4.21] [4.21 4.21 4.21] [0.785398, 0.61548, 7.291934]
[ 4.21  4.21 -4.21] [ 4.21  4.21 -4.21] [0.785398, -0.61548, 7.291934]
[ 4.21 -4.21  0.  ] [ 4.21 -4.21  0.  ] [-0.785398, 0.0, 5.953839]
[ 

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [112]:
centered_sph_coords[np.argsort(centered_sph_coords[:,2])]
np.array(structures[0].atomic_numbers)[np.argsort(centered_sph_coords[:,2])]

array([20, 20,  8,  8,  8, 12,  8, 12, 12, 12, 12, 12, 12, 12,  8, 12,  8,
       12,  8,  8,  8,  8,  8, 12,  8,  8, 12, 12, 12, 12, 12, 12, 12, 12,
        8,  8, 12,  8,  8,  8,  8,  8,  8,  8, 20, 20, 20, 20,  8, 20,  8,
        8,  8,  8,  8,  8,  8, 12, 20, 12, 12,  8,  8,  8, 12, 12, 12,  8,
       12,  8,  8,  8, 12, 12,  8,  8, 12,  8,  8, 12, 12, 12, 12, 12,  8,
        8,  8, 12, 20,  8,  8,  8,  8, 12,  8, 20, 20, 20, 20,  8,  8,  8,
       12,  8,  8,  8, 12, 12, 12,  8, 12, 12, 12, 12,  8, 12,  8,  8,  8,
        8,  8, 12,  8, 12,  8, 12, 12, 12, 12,  8, 12,  8,  8, 12, 12, 12,
       12, 12, 12, 12,  8,  8, 12,  8,  8,  8,  8,  8, 12,  8, 12,  8,  8,
       20, 20,  8,  8,  8, 12,  8,  8, 12, 12, 12,  8, 12,  8, 12, 12, 12,
       12,  8,  8,  8, 20, 20, 12,  8, 12,  8,  8,  8, 12, 12, 12,  8,  8,
        8,  8, 12, 12,  8,  8, 12,  8,  8, 12, 12,  8,  8, 12, 12, 20,  8,
       12,  8,  8, 12,  8, 12,  8,  8, 12, 12,  8, 12])

In [128]:
centered_sph_coords[:,0:2]

array([[ 0.      ,  0.      ],
       [ 0.      ,  1.570796],
       [ 0.      , -1.570796],
       [ 1.570796,  0.      ],
       [ 1.570796,  0.785398],
       [ 1.570796, -0.785398],
       [-1.570796,  0.      ],
       [-1.570796,  0.785398],
       [-1.570796, -0.785398],
       [ 0.      ,  0.      ],
       [ 0.      ,  0.785398],
       [ 0.      , -0.785398],
       [ 0.785398,  0.      ],
       [ 0.785398,  0.61548 ],
       [ 0.785398, -0.61548 ],
       [-0.785398,  0.      ],
       [-0.785398,  0.61548 ],
       [-0.785398, -0.61548 ],
       [ 3.141593,  0.      ],
       [ 3.141593,  0.785398],
       [ 3.141593, -0.785398],
       [ 2.356194,  0.      ],
       [ 2.356194,  0.61548 ],
       [ 2.356194, -0.61548 ],
       [-2.356194,  0.      ],
       [-2.356194,  0.61548 ],
       [-2.356194, -0.61548 ],
       [ 0.785398,  0.      ],
       [ 0.785398,  0.955317],
       [ 0.785398, -0.955317],
       [-1.249046,  0.      ],
       [-1.249046,  0.563943],
       [

In [129]:
np.unique(centered_sph_coords[:,0:2],axis=1,return_counts=True)

(array([[ 0.      ,  0.      ],
        [ 0.      ,  1.570796],
        [ 0.      , -1.570796],
        [ 1.570796,  0.      ],
        [ 1.570796,  0.785398],
        [ 1.570796, -0.785398],
        [-1.570796,  0.      ],
        [-1.570796,  0.785398],
        [-1.570796, -0.785398],
        [ 0.      ,  0.      ],
        [ 0.      ,  0.785398],
        [ 0.      , -0.785398],
        [ 0.785398,  0.      ],
        [ 0.785398,  0.61548 ],
        [ 0.785398, -0.61548 ],
        [-0.785398,  0.      ],
        [-0.785398,  0.61548 ],
        [-0.785398, -0.61548 ],
        [ 3.141593,  0.      ],
        [ 3.141593,  0.785398],
        [ 3.141593, -0.785398],
        [ 2.356194,  0.      ],
        [ 2.356194,  0.61548 ],
        [ 2.356194, -0.61548 ],
        [-2.356194,  0.      ],
        [-2.356194,  0.61548 ],
        [-2.356194, -0.61548 ],
        [ 0.785398,  0.      ],
        [ 0.785398,  0.955317],
        [ 0.785398, -0.955317],
        [-1.249046,  0.      ],
        